# Preamble

In [1]:
import sys
# pandas to manipulate SQL answer set
import pandas as pd
import time,os
# for Posgresql and other RDBMS
from sqlalchemy import create_engine,event,schema,Table,Column, Integer, Float, String, MetaData, TIMESTAMP, Date, text, inspect
from sqlalchemy_utils import database_exists,create_database
from sqlalchemy.orm import sessionmaker
from sqlalchemy.schema import CreateTable

## Connect to the DB server

In [4]:
# Define your PostgreSQL server connection details
database_name = 'redcross'
username = 'postgres'
password = 'postgres'
host = '127.0.0.1'  # or your server address
port = '5432'  # default PostgreSQL port


## Transaction to assign volunteers

In [34]:
import datetime
import psycopg2
conn = psycopg2.connect(f"dbname='{database_name}' user='{username}' host='{host}' password='{password}'")

request_id = 16

# --(15p) Create a transaction that will read valid applications for a request. Then
# --assigns the applicants as such:
# --- Prioritize the skills by their value of importance.
# --- Assign volunteers with valid applications and who have these skills
# --until the minimum number of volunteers needed for the skills is met
# --(assigning here means is_accepted gets TRUE, you may also create a
# --separate table volunteer_assignment that tracks request_id and
# --volunteer_id who got assigned to the request) (you may use your
# --scoring system for this)
# -- Assign the rest of applied volunteers.
# -- If the register by date is not past and the minimum number of
# --volunteers is not met (skill based or general), roll back.
# -- If the register by date is not past or the minimum number of
# --volunteers is met, commit the assignment.
# - - If the register by date is past and the minimum number of volunteers
# --is not met, either add more time to the register by date or accept the
# --volunteers.


with conn.cursor() as curs:
    curs.execute("""CREATE TABLE if not exists volunteer_assigned ( 
                assignment_id SERIAL PRIMARY KEY, 
                request_id INT,
                volunteer_id TEXT, 
                volunteer_skill text, 
                requested_skill text);""")

    conn.commit()

with conn.cursor() as curs:

    try:
        curs.execute("select number_of_volunteers from request where id= %s::int8", (request_id,))
        (vol_num_needed,) = curs.fetchone();
   
        curs.execute("""select skill_name, value as skill_value,min_need as skill_min_need from request_skill 
                     join request on request.id = request_skill.request_id  
                     where request.id = %s::int8
                     order by skill_value desc
                     """, (request_id,))
        enough_skills = True
        for skill_name, skill_value, skill_min_need in curs.fetchall():
            curs.execute("""
                        select count(*)
                        from volunteer_application va
                        join skill_assignment sa on sa.volunteer_id = va.volunteer_id
                        where request_id = %s::int8 and skill_name = %s and va.is_valid = TRUE and va.is_assigned = TRUE
                         """, (request_id, skill_name))
            (num_assigned,) = curs.fetchone()
            print('skill needed', skill_value, ' minimum with skill', skill_min_need, ' currently assigned ', num_assigned)
            curs.execute("""
                        select va.volunteer_id as volunteer_id, va.id as application_id
                        from volunteer_application va
                        join skill_assignment sa on sa.volunteer_id = va.volunteer_id
                        where request_id = %s::int8 and skill_name =%s and va.is_valid = TRUE and va.is_assigned = FALSE
                        """,(request_id, skill_name))
            for volunteer_id, application_id in curs.fetchall():
                print('volunteer_id', volunteer_id, 'application id', application_id)
                if num_assigned < skill_min_need:
                    curs.execute("""update volunteer_application set is_assigned=TRUE where id=%s""", (application_id,))
                    num_assigned = num_assigned + 1
                    print('assigning with skill', request_id, volunteer_id, skill_name )
                else: 
                    print ('enough assignments already, not assigning ', request_id, volunteer_id, skill_name )
            enough_skills = enough_skills and num_assigned >= skill_min_need
            print ('assigned with skill ', skill_name, ' assigned ', num_assigned, ' minimum ', skill_min_need)
            
        curs.execute("""
                    select count(*)
                    from volunteer_application va
                    where request_id = %s::int8 and va.is_valid = TRUE and va.is_assigned = TRUE
                    """,(request_id,))
        (num_assigned_total,) = curs.fetchone();

        curs.execute("""
                    select va.volunteer_id as volunteer_id, va.id as application_id
                    from volunteer_application va
                    where request_id = %s::int8 and va.is_valid = TRUE and va.is_assigned = FALSE
                    """,(request_id,))
        for volunteer_id, application_id in curs.fetchall():
            if num_assigned_total < vol_num_needed:
                num_assigned_total = num_assigned_total + 1
                curs.execute("""update volunteer_application set is_assigned=TRUE where id=%s""", (application_id,))
                print('assigning without skill', request_id, volunteer_id )
            else:
                break
        enough_volunteers = num_assigned_total >= vol_num_needed

        print ('assigning with request ', request_id, 'skills', enough_skills, enough_volunteers)        

        curs.execute("select register_by_date from request where id= %s::int8", (request_id,))
        (register_by_date,) = curs.fetchone();
        register_by_date_is_past = datetime.datetime.now() > register_by_date
        minimum_number_of_volunteers_is_met = enough_skills and enough_volunteers
        if (not register_by_date_is_past and not minimum_number_of_volunteers_is_met):
            print ('registration date is not past and minimum number of volunteers is not met, rollback')
            conn.rollback()
        elif (register_by_date_is_past and not minimum_number_of_volunteers_is_met):
            print ('registration date is past and minimum number of volunteers is not met, commit')
            conn.commit()
        else:
            conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

skill needed 5  minimum with skill 4  currently assigned  0
volunteer_id 170479-955C application id 842
assigning with skill 16 170479-955C EventHosting
assigned with skill  EventHosting  assigned  1  minimum  4
skill needed 2  minimum with skill 4  currently assigned  0
volunteer_id 240678-989B application id 2530
assigning with skill 16 240678-989B TeamGuide
assigned with skill  TeamGuide  assigned  1  minimum  4
assigning without skill 16 291069-904R
assigning without skill 16 160903A941P
assigning without skill 16 011074-9149
assigning without skill 16 120398-967M
assigning without skill 16 040804A913H
assigning with request  16 skills False False
registration date is not past and minimum number of volunteers is not met, rollback
